# Explore USC Course Progression Suggestions

Each course in a degree is given a 'course progression value' (CVP).
The next best courses to take are the ones in the current semester
that have the lowest 'course progression value'.

Course Progression Values (CPVs) are four-digit numbers in the form A.BCD (for example 3.251) where the A value relates to the Study Period (typically a semester) in the sequence of Study Periods in the optimal study plan, the B value relates to the course year level (100-, 200-, etc.), the C value relates to whether the course is required, recommended or elective (including in a major or minor), and the D value relates to whether the course is a pre-requisite for subsequent courses.

Course Progression Value Concept invented by: Graham Ashford

Author of this code: Mark Utting

TODO: 
* [DONE] read Excel files directly, instead of just *.csv.
* [DONE] read majors/minors from Excel file using Graham's layout.
* [DONE] if no semesters for each course, then just use CPV even/odd.
* allow a different load for each semester.
* show equal courses with '=' sign (to show student choice).
* discard XXX1nn electives (CPV=1.17x or 2.17x) after done>8 courses
* have special Elective1xx that only matches first-year electives
* threshold for allowing Elective2xx etc.
* check pre-reqs - where to read them from?
* add anti-reqs (as equivalent to required course)
* warn if exceed 10 first-year courses or 24/36 courses total?
* handle courses that are not 12 points.

## Parameters and Settings

In [1]:
ELECTIVE_PREFIX = "Elective"
LOAD = 4   # max courses each semester

In [2]:
import csv
import sys
from typing import Set, List, Dict

In [3]:
import pandas as pd
import numpy as np

## Code for Reading Student Records

In [103]:
class Student:
    """Basic student objects, to record id, name, courses they have passed, etc.
    Note: majors_minors is used for planning so should include the degree requirements.
    """
    def __init__(self, id:str, first:str, last:str, majors_minors:List[str]=[]):
        self.id = id
        self.last = last
        self.first = first
        self.majors_minors = majors_minors
        self.passed = set()
        
    def done(self, course_code:str, grade:str):
        # This currently adds every course, since we read the 'Clean Data' tab. 
        # TODO: check if passing grade? 
        # But include those in progress?
        self.passed.add(course_code)
        
    def __str__(self):
        return f"{self.id} {self.first} {self.last}"

In [104]:
def read_students(filename:str) -> List[Student]:
    """Read an Excel file of student results."""
    data = pd.read_excel(filename, sheet_name="Clean Data")
    students = []
    curr_stu = None # current student we are reading
    for i in data.index:
        if curr_stu == None or curr_stu.id != data["ID"][i]:
            # start new student
            curr_stu = Student(
                data["ID"][i],
                data["First Name"][i],
                data["Last"][i]
            )
            students.append(curr_stu)
        code = data["Subject"][i] + str(data["Catalog"][i])
        curr_stu.done(code, data["Grade"][i])
    return students

In [105]:
stu = read_students("Dummy student details.xlsx")
assert len(stu) == 2
assert stu[0].first == "Father"
assert stu[0].last == "Christmas"
assert len(stu[0].passed) == 8
assert "BUS101" in stu[0].passed
assert "ICT120" in stu[0].passed

## Code for Reading Programs / Majors / Minors with CPVs

In [106]:
class Course:
    """Simple course object, to record course code, title and progression value (cpv)."""
    def __init__(self, code, title, cpv):
        self.code = code
        self.title = title
        self.cpv = cpv
        
    def is_done(self, done:Set[str]) -> bool:
        # TODO: extend to handle anti-reqs?
        return self.code in done
    
    def is_elective(self, level:int=0):
        """True if this course is elective.
        The optional 'level' argument allows you to check if it is at a given year level.
        For example: is_elective(2) will be True for Elective201, False for Elective300.
        """
        level_str = ""
        if level > 0:
            level_str = str(level)
        return self.code.startswith(ELECTIVE_PREFIX + level_str)
    
    def __eq__(self, other):
        """Two courses are equal iff they have the same code."""
        if isinstance(other, Course):
            return self.code == other.code
        return False
    
    def __hash__(self):
        """Hash must be consistent with equals."""
        return hash(self.code)
    
    def __str__(self):
        return self.code
    
# Test Course objects
cor109 = Course("COR109", "Communication and Thought", 1.130)
ict221 = Course("ICT221", "Object-Oriented Programming", 3.130)
assert cor109 == cor109
assert ict221 != cor109
assert cor109 != 3

assert cor109.is_done(set()) == False
assert cor109.is_done(set(["COR109"])) == True

assert cor109.is_elective() == False
assert Course(ELECTIVE_PREFIX+"101", "", 1.130).is_elective() == True
assert Course(ELECTIVE_PREFIX+"101", "", 1.130).is_elective(1) == True
assert Course(ELECTIVE_PREFIX+"101", "", 1.130).is_elective(2) == False

In [107]:
def read_programs_lauren(excelfile:str) -> Dict[str,List]:
    """Reads an Excel file of programs (degrees) in Lauren's format.
    Each program/major/minor has a list of courses with CPVs.
    Required columns in Excel sheet:
    A. ignored
    B. Progression Value (or next degree+major name)
    C. Course Code
    D. Course Title
    E. Comment (optional)
    """
    sheet = pd.read_excel(excelfile, header=None)
    # These correspond to columns A,B,C,D,E,... in the Excel file
    sheet.columns = ["ignore", "CPV", "Code", "Title", "Comment"]
    programs = {}
    curr_prog = []
    for i in sheet.index:
        cpv = sheet.CPV[i]
        code = sheet.Code[i]
        title = sheet.Title[i]
        # print(cpv, code, title)
        if pd.notnull(cpv) and pd.isnull(code):
            print("reading", cpv)  # the name of the degree/major/minor
            curr_prog = [] # new list
            programs[cpv] = curr_prog
        elif code is str and code.lower() == "course code":
            pass  # ignore any column header rows
        elif isinstance(cpv, float) and pd.notnull(code) and pd.notnull(title):
            # print("    ", cpv, code, title)
            curr_prog.append(Course(code, title, cpv))
    return programs

In [108]:
# test that it works correctly
bict = read_programs_lauren("Course Progression BICT.xlsx")
assert len(bict) == 2
assert len(bict["BICT: Information Systems Major"]) == 24
assert len(bict["BICT: Web and Mobile Development Major"]) == 24

reading BICT: Information Systems Major
reading BICT: Web and Mobile Development Major


In [109]:
def read_programs_graham(excelfile:str) -> Dict[str,List]:
    """Reads an Excel file that defines programs/majors/minors.
    Assumes the file follows Graham's conventions: a single long
    list of courses, with column A having the program/major/minor
    name on the row where it starts.
    Each program/major/minor has a list of courses with CPVs.
    Required columns in Excel sheet:
    A. Program/Major/Minor name (on the row where it starts)
    B. Progression Value
    C. Course Code
    D. Course Title
    E. Comment (optional)
    """
    sheet = pd.read_excel(excelfile)
    # Define the column names in the Excel file
    column_name = "Major/Minor"
    column_cpv = "Progression value"
    column_code = "Course code"
    column_title = "Course title"
    degrees = {} # maps each component name to a list of its courses
    courses = []
    for i in sheet.index:
        name = sheet[column_name][i]
        cpv = sheet[column_cpv][i]
        code = sheet[column_code][i]
        title = sheet[column_title][i]
        # print(name, cpv, code, title)
        if pd.notnull(name):
            # start a new program/major/minor 
            print("Reading:", name)
            courses = []
            degrees[name] = courses
        if isinstance(cpv, float) and pd.notnull(code) and pd.notnull(title):
            courses.append(Course(code, title, cpv))
    return degrees

In [110]:
# test that it works correctly
bsc = read_programs_graham("Course Progression BSc.xlsx")
bsc_majors = [m for m in bsc.keys() if m.endswith("major")]
bsc_minors = [m for m in bsc.keys() if m.endswith("minor")]
assert len(bsc_majors) == 5
assert len(bsc_minors) == 11
assert len(bsc["BSc"]) > 10
assert len(bsc["Chemistry minor"]) == 4

Reading: BSc
Reading: Biology major
Reading: Biotechnology major
Reading: Chemisty major
Reading: Ecology major
Reading: Mathematics major
Reading: Biology minor
Reading: Biochemistry minor
Reading: Biotechnology minor
Reading: Chemistry minor
Reading: Agriculture and Aquaculture minor
Reading: Genetics minor
Reading: Mathematics minor
Reading: Microbiology minor
Reading: Sustainability minor
Reading: Journalism minor
Reading: Entrepreneurship minor
Reading: Other pre-reqs not included in majors or minors


## Code for recommending which courses students should take

In [111]:
def level(code:str) -> int:
    """Return the year-level of a given course code."""
    if code.startswith(ELECTIVE_PREFIX):
        return int(code[len(ELECTIVE_PREFIX)])
    else:
        return int(code[3])

# Test this function.
assert level("ABC234") == 2
assert level(ELECTIVE_PREFIX + "321") == 3

In [112]:
def is_allowed(course:Course, done:Set[str], semester:int) -> bool:
    """True if the given course (code) has not been done,
    and it is allowed to be taken in this semester (the even/odd trick)
    and if it is a level 100 elective then student has done < 8 courses
    and if it is a level 100 elective then student has done < 16 courses.
    """
    correct_semester = (int(course.cpv) % 2) == (semester % 2)
    #ignore100 = course.code.startswith(ELECTIVE_PREFIX + "1") and len(done) >= 8
    #ignore200 = course.code.startswith(ELECTIVE_PREFIX + "2") and len(done) >= 2 * 8
    return course.code not in done and correct_semester # and not ignore100 and not ignore200

# Test this function:
abc1 = Course("ABC110", "News Science", 1.230)
abc2 = Course("ABC110", "News Science", 2.230)
abc3 = Course("ABC110", "News Science", 3.230)
abc4 = Course("ABC110", "News Science", 4.230)
assert is_allowed(abc1, set(), 1) == True
assert is_allowed(abc2, set(), 1) == False
assert is_allowed(abc3, set(), 1) == True
assert is_allowed(abc4, set(), 1) == False

assert is_allowed(abc1, set(), 2) == False
assert is_allowed(abc2, set(), 2) == True
assert is_allowed(abc3, set(), 2) == False
assert is_allowed(abc4, set(), 2) == True

assert is_allowed(abc1, set(["ABC123"]), 2) == False

def eight(year:int) -> List[str]:
    return [Course(f"ABC{year}2{i}", "Title", 2 * year + 0.230) for i in "12345678"]

assert is_allowed(Course("Elective101", "", 4.0), set([]), 2) == True
#assert is_allowed(Course("Elective101", "", 2.0), set(eight(1)), 2) == False
assert is_allowed(Course("Elective201", "", 4.0), set([abc1]), 2) == True
#assert is_allowed(Course("Elective201", "", 2.0), set(eight(1) + eight(2)), 2) == False

In [113]:
def pretty(codes:Set[str]) -> str:
    """Pretty-print a set of course codes into a string."""
    return " ".join(sorted(list(codes)))

# Test this function.
assert pretty(set(["ABC323", "ABC100"])) == "ABC100 ABC323"

In [114]:
def plan_student_old(stu:Student, programs:Dict[str,List], output=sys.stdout):
    """Print all remaining courses for this student, by semester."""
    done = stu.passed
    output.write(f"{stu.id} {stu.first} {stu.last} {stu.program}\n")
    progression = sorted(programs[stu.program])  # sort by progression code
    required_courses = set([c for (p,c,t,s) in progression]) # includes electives
    required_electives = set([c for c in required_courses if c.startswith(ELECTIVE_PREFIX)])
    # partition 'done' into three subsets
    done_required = done.intersection(required_courses)
    done_extra = done.difference(required_courses)
    done_electives = list(done_extra)[0:len(required_electives)]
    done_extra = done_extra.difference(done_electives)
    assert done_required.union(done_electives).union(done_extra) == done
    required_electives_done = sorted(list(required_electives))[0:len(done_electives)]
    done = done.union(required_electives_done)
    output.write("    done required: " + pretty(done_required) + "\n")
    output.write("    done electives:" + pretty(done_electives) + "\n")
    # output.write("    as electives : " + pretty(required_electives_done) + "\n")
    if done_extra:
        output.write("    WASTED :-(   : " + pretty(done_extra) + "\n")
    
    # now spread the remaining courses out over several semesters
    remaining = [(p,c,t,s) for (p,c,t,s) in progression if c not in done]
    sem = START_SEMESTER
    while remaining:
        this_sem = [c for (p,c,t,s) in remaining if s == sem]
        do_now = this_sem[0:LOAD]
        do_now_string = " ".join(do_now)
        output.write(f"    sem{sem}: {do_now_string}\n")
        # update done and remaining, then move to next semester
        done = done.union(set(do_now))
        remaining = [(p,c,t,s) for (p,c,t,s) in remaining if c not in done]
        if sem == 1:
            sem = 2
        else:
            sem = 1

In [115]:
def whole_program(programs:Dict[str,List], majors_minors:List[str]) -> List[Course]:
    """Expand a degree name plus majors and minors into one total list of requirements."""
    progression = sum([programs[m] for m in majors_minors], [])
    return sorted(progression, key=lambda c: c.cpv) # then sort by CPV

In [116]:
def remove_done(progression, done:Set[str]) -> List[Course]:
    """Remove courses that are satisfied by the 'done' set (of course codes)."""
    return [c for c in progression if not c.is_done(done)]

In [123]:
def allocate_elective(elective:Course, done:Set[str]) -> str:
    """Choose a course from 'done' for this elective, else return None."""
    for code in sorted(list(done), key=lambda c: c[3:]):
        # if level(code) >= level(elective.code):
            return code
    return None

# Test this function.
e = Course(ELECTIVE_PREFIX + "200", "", 2.341)
assert allocate_elective(e, set([])) == None
#assert allocate_elective(e, set(["ABC123"])) == None
assert allocate_elective(e, set(["ABC123", "ABC234"])) == "ABC123"
#assert allocate_elective(e, set(["ABC123", "ABC234"])) == "ABC234"
assert allocate_elective(e, set(["ABC323", "ABC234", "ABC333"])) == "ABC234" # the lowest level one

In [133]:
def finished(progression, done:Set[str]) -> bool:
    """Student is finished if they have only electives left, and have done enough courses."""
    return len(done) >= 24 and all([c.is_elective() for c in progression])

In [138]:
def plan_student(stu:Student, progression:List[Course], semester:int, output=sys.stdout):
    """Print all remaining courses for this student, by semester."""
    # step 1: tick off all required courses already done
    required_codes = set([c.code for c in progression])
    done = stu.passed.intersection(required_codes)
    done_extra = stu.passed.difference(done) # these may be used as electives
    progression = remove_done(progression, done)
    output.write(f"    done: {done}\n")
    if done_extra:
        output.write(f"    extra {done_extra}\n")
        
    # step 2: loop through the current and future semesters
    # Note: we allocate the 'done_extra' courses to electives as we go.
    timeout = 0
    while not finished(progression, done) or timeout > 10:
        todo = set()
        for course in progression:
            if is_allowed(course, done, semester):
                if course.is_elective():
                    e = allocate_elective(course, done_extra)
                    if e != None:
                        # satisfy this elective by the course they have already done
                        done.add(e)
                        done_extra.remove(e)
                        print(f"          {course.code} satisfied by {e}")
                    elif len(done) < 8 * level(course.code):         # too restrictive ??? 
                        # get them to do this elective
                        todo.add(course)
                        done.add(course.code)
                else:
                    todo.add(course)
                    done.add(course.code)
                # see if this semester is full?
                left = [c for c in progression if c not in todo]
                if len(todo) == LOAD or finished(left, done):
                    break
        todo_codes = [c.code for c in todo]
        output.write(f"    sem{semester}: {pretty(todo_codes)}\n")
        progression = [c for c in progression if c not in todo]
        # move to next semester
        timeout += 1
        if semester == 1:
            semester = 2
        else:
            semester = 1

    if done_extra:
        output.write("    WASTED :-(   : " + pretty(done_extra) + "\n")
    output.write(f"    Total courses done: {len(done)}\n\n")

### Example BICT Students

In [144]:
# Some BICT test cases (choose one of the following majors)
bict_is = whole_program(bict, ["BICT: Information Systems Major"])
bict_wm = whole_program(bict, ["BICT: Web and Mobile Development Major"])

year1 = ["ICT110", "ICT112", "ICT115", "ICT120", "COR109", "BUS104", "BUS106", "BUS101"]
year2 = ["ICT211", "BUS203", "DES105", "ICT220", "BUS211", "ICT321"]

s1 = Student("0000000", "New", "Student")
s1.passed = set()

s2 = Student("0000000", "Second-Year ICT", "Student")
s2.passed = set(year1)

s3 = Student("0000000", "Third-Year IS", "Student")
s3.passed = set(year1+year2)

s4 = Student("0000000", "Vacilating", "Student")
s4.passed = set(year1 + ["ABC20"+c for c in "12345678"])

for s in [s1,s2,s3,s4]:
    for start_semester in [1,2]:
        print(f"{s} BICT Web & Mobile major, start semester {start_semester}")
        plan_student(s, bict_wm, start_semester)

0000000 New Student BICT Web & Mobile major, start semester 1
    done: set()
    sem1: BUS104 COR109 ICT110 ICT112
    sem2: BUS101 ENG103 ICT115 ICT120
    sem1: DES221 ICT211 ICT220 ICT221
    sem2: DES222 Elective200 ICT311 ICT320
    sem1: CSC301 Elective300 ICT310 ICT352
    sem2: Elective301 Elective302 ICT342 ICT351
    Total courses done: 24

0000000 New Student BICT Web & Mobile major, start semester 2
    done: set()
    sem2: BUS101 ENG103 ICT115 ICT120
    sem1: BUS104 COR109 ICT110 ICT112
    sem2: DES222 Elective200 ICT311 ICT320
    sem1: DES221 ICT211 ICT220 ICT221
    sem2: Elective301 Elective302 ICT342 ICT351
    sem1: CSC301 Elective300 ICT310 ICT352
    Total courses done: 24

0000000 Second-Year ICT Student BICT Web & Mobile major, start semester 1
    done: {'BUS104', 'ICT120', 'ICT112', 'ICT110', 'COR109', 'BUS101', 'ICT115'}
    extra {'BUS106'}
    sem1: DES221 ICT211 ICT220 ICT221
    sem2: DES222 ENG103 ICT311 ICT320
          Elective300 satisfied by BUS10

### Example BSc Students

In [146]:
start_semester = 1
prog = whole_program(bsc, ["BSc", "Biology major", "Biology minor"])
s = Student("0000000", "New science", "Student")
plan_student(s, prog, start_semester)

    done: set()
    sem1: COR109 LFS100 SCI110 SCI113
    sem2: Elective102 LFS112 SCI102 SCI105
    sem1: ANM203 ENS221 LFS261 SCI202
    sem2: ENS222 Elective202 LFS252 SCI212
    sem1: BIM331 Elective300 Elective301 MBT301
    sem2: ENS318 ENS330 Elective302 Elective303
    sem1: 
    sem2: ENS333 SCI302 SUS310 WPL310
    sem1: 
    sem2: SRP301
    Total courses done: 29



In [145]:
# Analyse all combinations of 1 major + 1 minor.
start_semester = 1
for major in bsc_majors:
    for minor in bsc_minors:
        prog = whole_program(bsc, ["BSc",major,minor])
        s = Student("0000000", "BSc", "Student")
        print(f"---- BSc + {major} + {minor} ----")
        plan_student(s, prog, start_semester)


---- BSc + Biology major + Biology minor ----
    done: set()
    sem1: COR109 LFS100 SCI110 SCI113
    sem2: Elective102 LFS112 SCI102 SCI105
    sem1: ANM203 ENS221 LFS261 SCI202
    sem2: ENS222 Elective202 LFS252 SCI212
    sem1: BIM331 Elective300 Elective301 MBT301
    sem2: ENS318 ENS330 Elective302 Elective303
    sem1: 
    sem2: ENS333 SCI302 SUS310 WPL310
    sem1: 
    sem2: SRP301
    Total courses done: 29

---- BSc + Biology major + Biochemistry minor ----
    done: set()
    sem1: COR109 LFS100 SCI110 SCI113
    sem2: Elective102 LFS112 SCI102 SCI105
    sem1: ANM203 LFS251 LFS261 SCI202
    sem2: ENS222 LFS252 MLS211 SCI212
    sem1: BIM331 BIM341 ENS221 MBT301
    sem2: ENS318 Elective302 Elective303 NUT232
    sem1: 
    sem2: ENS330 ENS333 SCI302 WPL310
    sem1: 
    sem2: SRP301 SUS310
    Total courses done: 30

---- BSc + Biology major + Biotechnology minor ----
    done: set()
    sem1: COR109 LFS100 SCI110 SCI113
    sem2: Elective102 LFS112 SCI102 SCI105
    

## Analyse some real students

In [20]:
stu_bict = read_students("BICT student details.xlsx")
print(f"Planning {len(stu_bict)} BICT students.")
major = BICT_WM.replace(":", " ")
start_semester = 1
with open(f"Report {major} Start Semester {start_semester}.txt", "w") as output:
    for s in stu_bict:
        s.program = BICT_WM
        plan_student(s, bict, start_semester, output)

Planning 119 BICT students.
